# Imports
The code imports several libraries to work correctly. The libraries are as follows:

pickle: This module implements binary protocols for serializing and de-serializing a Python object structure. It is used here to save the metadata information in binary format.

os: This module provides a portable way of using operating system dependent functionality like reading or writing to the file system. It is used here to join paths and create directories.

zipfile: This module allows us to read and write ZIP archive files. It is used here to extract images from a compressed archive.

requests: This module allows us to send HTTP/1.1 requests using Python. It is used here to download images from a URL.

functools: This module provides various functions that can be used to create higher-order functions. It is used here to define the cached_property decorator.

pathlib: This module provides a higher-level interface to working with file system paths than the os module. It is used here to create directories.

tqdm: This module provides a progress bar that can be used to track the progress of a long-running operation.

json: This module provides methods for working with JSON data. It is used here to save the metadata information in JSON format.

sqlite3: This module provides a Python interface to SQLite databases. It is used here to create and interact with a SQLite database.

pandas: This module is used for data manipulation and analysis. It is used here to read CSV files and create a pandas DataFrame.

PIL: This module provides an interface for opening, manipulating, and saving many different image file formats. It is used here to extract metadata from images.

nest_asyncio: This module provides a way to run nested event loops in asyncio.

asyncio: This module provides infrastructure for writing single-threaded concurrent code using coroutines, multiplexing I/O access over sockets and other resources, running network clients and servers, and other related primitives.

aiohttp: This module provides an asynchronous HTTP client/server implementation using asyncio.

time: This module provides various time-related functions. It is used here to measure the time taken to download images.

subprocess: This module provides a way to spawn new processes, connect to their input/output/error pipes, and obtain their return codes. It is used here to check if the exiftool command line tool is installed.

tqdm_asyncio: This module provides a progress bar for asyncio applications. It is used here to track the progress of processing all images.

In [1]:
!pip install requests pathlib tqdm pandas pillow nest_asyncio aiohttp mysql-connector-python python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [ ]:
import os
import zipfile
import requests
import functools
import pathlib
from tqdm import tqdm
import pandas as pd
import nest_asyncio
import aiohttp
import time
import asyncio
import mysql.connector
from mysql.connector import Error
import csv
from dotenv import load_dotenv
load_dotenv()

# Settings base variables and paths
For this project, we used the unsplash dataset, which is a large-scale image dataset. The dataset contains over 25,000 images.
The code sets the base variables and paths for the project. The variables are as follows:

In [ ]:
# Set the base folder path for the project
output_path = "../output"
images_path = os.path.join(output_path, "images")
metadata_path = os.path.join(output_path, "metadata")
include_path = os.path.join(output_path, "include")

list_of_paths = [output_path, images_path, metadata_path, include_path]

# Set the base URL for the dataset
dataset_url = "https://unsplash.com/data/lite/latest"
# metadata mode (used to save metadata)
metadata_mode = "sqlite"

# Set the number of images to download
num_images = 1000

# Set SQL variables
sql_host = os.getenv("SQL_HOST")
sql_user = os.getenv("SQL_USER")
sql_password = os.getenv("SQL_PASSWORD")
sql_database = os.getenv("SQL_DATABASE")

# Create folder structure
The code creates the folder structure for the project. The folder structure is as follows:
- output
    - images
    - metadata
    - config

This method creates a folder with the given path if it doesn't already exist, It also outputs a message to inform the user if the folder was created or if it already exists.
This is useful for organizing and managing files in a project. By creating a folder to store data and resources, it keeps the working directory tidy and makes it easier to locate files. Additionally, by checking if the folder exists before creating it, it prevents the program from overwriting existing data or throwing an error.

In [ ]:
def create_folder(path):
    """
    This function creates a folder at the specified path.
    If the folder already exists, it will print a message saying so.
    If there is an error creating the folder, it will print the error message.

    Parameters:
        :param path (str): The path of the folder to be created.

    Returns:
    None
    """
    try:
        # Use os.mkdir to create the folder at the specified path
        os.mkdir(path)
        print(f"Folder {path} created")
    except FileExistsError:
        # If the folder already exists, print a message saying so
        print(f"Folder {path} already exists")
    except Exception as e:
        # If there is an error creating the folder, print the error message
        print(f"Error creating folder {path}: {e}")

# Create the folder structure
This method initializes a list of folders by calling the create_folder method for each folder in the list.
The purpose of this method is to make sure that all necessary folders exist before the program continues its execution.
If a folder does not exist, the create_folder method will create it. If a folder already exists, the method will simply print a message indicating that the folder already exists. In case of any other error, the method will print the error message.

In [ ]:
def init_folder(folder_names: list):
    for folder_name in folder_names:
        create_folder(folder_name)

In [ ]:
init_folder(list_of_paths)

# Define methods for downloading the dataset
The following code block is a method to download a file from a given URL and save it to a specified filename.

The method starts by creating a session (s = requests.Session()) and then mounting it to the URL (s.mount(url, requests.adapters.HTTPAdapter(max_retries=3))). This sets the maximum number of retries to 3 if the connection to the URL fails.

Then, the method makes a GET request to the URL (r = s.get(url, stream=True, allow_redirects=True)) and checks if it returns a successful response (r.raise_for_status()). If there was an HTTP error during the request, the error message is printed (print(f"HTTP error occurred while downloading dataset: {e}")).

The method also checks the file size specified in the response headers and assigns it to the variable file_size (file_size = int(r.headers.get('Content-Length', 0))). If the file size is 0, a default value of "(Unknown total file size)" is assigned to the variable desc; otherwise, the variable desc is left empty.

Next, the method resolves the file path and creates a directory if it doesn't already exist (path.parent.mkdir(parents=True, exist_ok=True)). The method then creates a tqdm progress bar to show the download progress (with tqdm.tqdm(total=file_size, unit='B', unit_scale=True, desc=desc) as pbar:).

Finally, the method writes the contents of the file to disk in chunks (for chunk in r.iter_content(chunk_size=1024):), updating the progress bar for each chunk that is written to disk (pbar.update(len(chunk))). If an error occurred during the download, a message with the error is printed (print(f"Error occurred while downloading dataset: {e}")). The file path is returned when the method is finished.

In [ ]:
def download(url, filename):
    """
    This download a file from a given URL and save it to a specified filename.

    Parameters:
        :param url (str): The URL of the file to be downloaded.
        :param filename (str): The filename to save the file as.

    Returns:
    path (str): The path of the downloaded file.
    """
    try:
        # Create a session object to persist the state of connection
        s = requests.Session()
        # Retry connecting to the URL up to 3 times
        s.mount(url, requests.adapters.HTTPAdapter(max_retries=3))
        # Send a GET request to the URL to start the download
        r = s.get(url, stream=True, allow_redirects=True)
        # Raise an error if the response is not 200 OK
        r.raise_for_status()
        # Get the file size from the Content-Length header, default to 0 if not present
        file_size = int(r.headers.get('Content-Length', 0))
        # Get the absolute path to the target file
        path = pathlib.Path(filename).expanduser().resolve()
        # Create parent directories if they don't exist
        path.parent.mkdir(parents=True, exist_ok=True)
        # Set the description to display while downloading, "(Unknown total file size)" if file size is 0
        desc = "(Unknown total file size)" if file_size == 0 else ""
        # Enable decoding the response content
        r.raw.read = functools.partial(r.raw.read, decode_content=True)
        # Use tqdm to display the download progress
        with tqdm(total=file_size, unit='B', unit_scale=True, desc=desc) as pbar:
            # Open the target file in binary write mode
            with path.open("wb") as f:
                # Write each chunk of data from the response to the file
                for chunk in r.iter_content(chunk_size=1024):
                    f.write(chunk)
                    pbar.update(len(chunk))
        # Return the path to the downloaded file
        return path
    # Handle HTTP error if the response is not 200 OK
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred while downloading dataset: {e}")
    # Handle any other exceptions that might occur while downloading the file
    except Exception as e:
        print(f"Error occurred while downloading dataset: {e}")

# Download the dataset
The following code block downloads the dataset from the URL and saves it to the specified filename. The method also prints a message to inform the user that the download is complete.

In [ ]:
def download_dataset(dataset_url, image_path):
    """
    Downloads the dataset from the given URL, unzips it, and stores the images in the specified image path.

    Args:
        :param dataset_url (str): URL of the dataset to be downloaded
        :param image_path (str): Path to store the images after unzipping the dataset
    """
    # Check if the dataset has already been downloaded
    # Check if the archive.zip file exists or if the images folder is empty
    if not os.path.exists('archive.zip'):
        # Download the dataset from the given url
        download(dataset_url, 'archive.zip')
        print("Dataset downloaded!")
        try:
            # Extract the contents of the archive.zip to the specified image path
            with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
                zip_ref.extractall(image_path)
            print("Dataset unzipped")
        except Exception as e:
            print(f"Error occurred while unzipping dataset: {e}")
        try:
            # Remove the archive.zip file
            os.remove('archive.zip')
            print("archive.zip removed")
        except Exception as e:
            print(f"Error occurred while removing archive.zip: {e}")

In [ ]:
download_dataset(dataset_url, images_path)

Allow the notebook to run asynchronously

In [ ]:
nest_asyncio.apply()

In [ ]:
# Read photo.tsv file in images folder
photo_df = pd.read_csv(os.path.join(images_path, 'photos.tsv000'), sep='\t')
# read photo_image_url column and photo_id in index
photo_df = photo_df[['photo_id', 'photo_image_url']]

print(photo_df.head())

This method downloads an image from a given URL using an asynchronous HTTP client library called aiohttp. The downloaded image is saved to a file on the local file system with a filename in the format "image_#index.jpg", where #index is a given integer value.

The method takes four arguments:

session: an instance of an aiohttp client session that manages HTTP requests and responses.
url: a string representing the URL of the image to download.
i: an integer representing the index of the image to download.
err_cnt: an optional integer representing the number of times that the download has failed due to a client error. If this is not provided, it defaults to 0.
The method first checks whether an error count was provided, and if not, sets it to 0. It then attempts to download the image using the aiohttp session.get() method, which returns a response object. The async with statement ensures that the response is properly handled and that the connection to the server is closed when the request is completed.

Once the response is obtained, the method constructs a filename using the given index value, and writes the image content to a file with that filename in binary mode. It then prints a message indicating that the image was downloaded and saved to the specified filename.

If the download fails due to a client error (e.g., a network timeout), the method catches the error using an except block, prints an error message indicating the URL that failed and the error that occurred, and then waits for 10 seconds before retrying the download. If the error count reaches 10, the method returns without attempting to download the image again.

If the download fails due to a server error (e.g., a 404 Not Found response), the exception is not caught and will propagate up the call stack.

If the download is successful, the method returns nothing.

In [ ]:
async def download_image(session: aiohttp.ClientSession, url: str, i: int, err_cnt=None):
    """
    Downloads an image from the given URL using an aiohttp client session and saves it to the local file system.

    Args:
        session: An aiohttp client session that manages HTTP requests and responses.
        url: The URL of the image to download.
        i: An integer representing the index of the image to download.
        err_cnt: An optional integer representing the number of times that the download has failed due to a client error.
                 If not provided, it defaults to 0.

    Raises:
        This method does not raise any exceptions.

    Returns:
        None.
    """
    if err_cnt is None:
        err_cnt = 0
    try:
        async with session.get(url) as response:
            filename = os.path.join(images_path, "image_" + str(i) + ".jpg")
            with open(filename, 'wb') as f:
                f.write(await response.content.read())
            print(f"Downloaded {url} to {filename} idx: {i}")
    except aiohttp.ClientError as e:
        print(f"Error occurred while downloading {url}: {e}")
        if err_cnt == 10:
            return
        await asyncio.sleep(10)
        err_cnt += 1
        await download_image(session, url, i, err_cnt)

This method, download_images, is a function that downloads a list of images from the web using the aiohttp library and saves them to the local file system. The method takes two arguments: image_urls, a list of URLs where the images are hosted, and images_ids, a list of identifiers for the images that will be used to name the files when they are saved locally.

The method starts by creating an aiohttp.ClientSession object, which is used to manage HTTP requests and responses. It then initializes an empty list tasks and creates a semaphore object with a maximum value of 5000. The semaphore is used to limit the number of concurrent downloads and prevent overloading the server.

Next, the method loops through the image_urls list using the built-in enumerate function to keep track of the index of each URL. For each URL, the method tries to acquire a permit from the semaphore to start a new download. If the maximum number of concurrent downloads has been reached, the method blocks until a permit becomes available.

Once a permit is acquired, the method creates a new task using the asyncio.ensure_future function and adds it to the tasks list. The task represents the asynchronous download of the image from the current URL using the download_image method. A callback function is also added to the task that releases the semaphore permit when the task completes, so that another download can start.

If an error occurs while trying to download an image, the method prints an error message and releases the semaphore permit. The method then continues with the next URL in the list.

After all tasks have been created and added to the tasks list, the method waits for them to complete using the asyncio.wait function. Finally, it gathers all the completed tasks using the asyncio.gather function, which ensures that all tasks have finished before the method returns.

Overall, the download_images method is an efficient and asynchronous way to download a large number of images from the web and save them to the local file system.

In [ ]:
async def download_images(image_urls, images_ids):
    """
    Downloads a list of images from the given URLs using an aiohttp client session and saves them to the local file system.

    Args:
        image_urls: A list of strings representing the URLs of the images to download.
        images_ids: A list of integers representing the indices of the images to download.

    Raises:
        This method does not raise any exceptions.

    Returns:
        None.
    """
    # Create a new aiohttp client session to manage HTTP requests and responses
    async with aiohttp.ClientSession() as session:
        tasks = []  # Create an empty list to hold the tasks that will download the images
        semaphore = asyncio.Semaphore(5000)  # Create a semaphore to limit the number of concurrent downloads
        # Loop through the image URLs and create a new task for each one
        for i, url in enumerate(image_urls):
            try:
                await semaphore.acquire()  # Acquire a permit from the semaphore to limit concurrency
                #url = url + "?w=1000&fm=jpg&fit=max"  # Append query parameters to resize and optimize the image
                task = asyncio.ensure_future(download_image(session, url, images_ids[i]))  # Create a new download task
                task.add_done_callback(
                    lambda x: semaphore.release())  # Release the semaphore permit when the task completes
                tasks.append(task)  # Add the task to the list of download tasks
            except Exception:
                print(f"Error occurred while downloading {url}")
                semaphore.release()  # Release the semaphore permit if an exception occurs
        # Wait for all download tasks to complete
        await asyncio.wait(tasks)
        # Gather the results of all download tasks (not necessary because the tasks have already completed)
        await asyncio.gather(*tasks)

In [ ]:
# Get the list of image urls and image ids
image_urls = photo_df['photo_image_url'].values.tolist()[:num_images]
# img id are from 0 to size of the list
images_ids = [i for i in range(len(image_urls))][:num_images]
# filter by looking if the image already exist in fact of the image_id is already in the folder
# Loop on the image_id and check if the image exist in the folder
image_urls = [url for url, image_id in zip(image_urls, images_ids) if
              not os.path.exists(os.path.join(images_path, "image_" + str(image_id) + ".jpg"))]
print(f"Number of images to download: {len(image_urls)}")

In [ ]:
# Split the list of image urls into chunks of max and add a timeout of 30 seconds
chunks = [image_urls[i:i + 5000] for i in range(0, len(image_urls), 5000)]
start_t = time.time()
loop = None
for i, chunk in enumerate(chunks):
    start = time.time()
    try:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(download_images(chunk, images_ids[i * 5000:(i + 1) * 5000]))
    except Exception as e:
        print(f"Error occurred while downloading chunk {i}: {e}")
    finally:
        loop.close()
        print(f"[Chunk {i}] Downloaded {len(chunk)} images in {time.time() - start} seconds")

print(f'Downloaded {len(image_urls)} images in {time.time() - start_t} seconds')

Clear the images folder from all files except images

In [ ]:
# Remove all files except images
for file in os.listdir(images_path):
    if file.endswith('.jpg'):
        continue
    else:
        try:
            # Don't delete TERMS.md
            if file == 'TERMS.md':
                continue
            os.remove(os.path.join(images_path, file))
        except Exception as e:
            continue

# Define methods to get all the image paths
The get_all_images method is used to retrieve all images present in the specified image path. It uses the os.walk function to traverse through all subdirectories within the image path and collects the file names that end with either '.png' or '.jpg' extensions. The full path of each image is then generated by joining the root directory and the file name. The method returns a list of all images' full paths. In case of any error, an error message is printed and an empty list is returned.

In [ ]:
def get_all_images(path):
    """Get all images from the given path.

    Args:
    param: image_path (str): path to the directory containing the images.

    Returns:
    - list: a list of full path to all the images with png or jpg extensions.
    - empty list: an empty list if an error occurred while fetching images.
    """
    try:
        # use os.walk to traverse all the subdirectories and get all images
        return [os.path.join(root, name)
                for root, dirs, files in os.walk(path)
                for name in files
                if name.endswith((".png", ".jpg"))]
    except Exception as e:
        # return an empty list and log the error message if an error occurred
        print(f"An error occurred while fetching images: {e}")
        return []

# Define methods to get metadata
The goal of the get_metadata method is to extract metadata information from a list of image files and return it in a dictionary format. This method uses the ExifTool software to extract the metadata information from the images. The input parameter is a string containing all image file paths separated by a space. The output of the method is a dictionary containing the metadata information of the images. If an error occurs for any image, the metadata for that image will be None. This method is implemented as an asynchronous coroutine using the asyncio module in Python.


This function takes a list of dictionaries containing metadata information of images and generates a list of SQL requests to insert the metadata into a database.

The function first creates an empty list to store the SQL requests. It then loops over each metadata dictionary in the input list using the tqdm function to provide a progress bar. For each metadata dictionary, the function extracts the filename of the image and then loops over all the items in the dictionary.

For each key-value pair in the metadata dictionary, the function creates an SQL request to insert the metadata into the database. The SQL request is in the form of a string that contains the filename, key, and value of the metadata item. The function adds each SQL request to the list of SQL requests.

After processing all the metadata dictionaries, the function returns the list of SQL requests.

In [ ]:
def gen_sql_requests(metadatas):
    """
    This function generates a list of SQL requests to insert metadata into a database.

    Parameters:
    metadatas (list): A list of dictionaries containing the metadata information of the images.

    Returns:
    list: A list of SQL requests to insert metadata into a database.
    """
    # Create a list to store SQL requests
    sql_requests = []
    # Loop over all metadata
    for i, metadata in enumerate(metadatas):
        try:
            # Get the filename of the image
            filename = metadatas[i]['filename']

            # Loop over all metadata items
            for key, value in metadatas[i].items():
                # Create SQL request to insert metadata into database
                # replace " by space
                value = value.replace('"', ' ')
                # replace ' by space
                value = value.replace("'", ' ')

                sql_request = f"INSERT INTO metadata VALUES ('{filename}', '{key}', '{value}')"
                # Add SQL request to list
                sql_requests.append(sql_request)

        except Exception as e:
            # Print an error message if an error occurs
            print("An error occurred while generating SQL requests: ", e)
            continue
    # Return the list of SQL requests
    return sql_requests

This method executes a SQL query on a SQLite database. The method takes a single parameter, queries, which should be a valid SQL queries that is compatible with the SQLite database.

The method first establishes a connection to the SQLite database using the connect() method of the sqlite3 module in Python. It then executes the SQL query using the execute() method of the connection object. After executing the query, the method commits the changes to the database using the commit() method of the connection object, and then closes the connection using the close() method of the connection object.

This method is typically used to insert metadata information into a SQLite database. It assumes that the SQLite database already exists and is located in the metadata_path directory.

In [ ]:
def create_server_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

This is an asynchronous coroutine get_all_metadata that extracts metadata from all images in a directory and saves the metadata information in either pickle or JSON format. The function takes two parameters: image_path which is the path to the directory where the images are stored, and metadata_path which is the path to the directory where the metadata will be saved.

Firstly, the function retrieves a list of all images in the directory using the get_all_images method. It then creates a Semaphore object with a limit of 5000 to limit the number of simultaneous coroutines to 5000.

A progress bar is created using the tqdm_asyncio library to track the progress of processing all images. A list of coroutines is created using a list comprehension with each coroutine being an instance of the get_metadata method applied to each image file in the directory.

The coroutines are then executed concurrently using the asyncio.as_completed method, with a maximum of 5000 coroutines being executed at a time, and their results are appended to a metadatas list. For each successfully extracted metadata, a SQL query is generated and executed to insert the metadata into the database using the gen_sql_requests and execute_query functions.

Once all coroutines are completed, the metadata information is saved into the database. If an error occurs during the metadata extraction process, the None value is returned for that image and the error message is printed.

In [ ]:
async def get_all_metadata(images_path):
    """
    This coroutine extracts metadata from all images in a directory and saves the metadata information in either pickle or json format.

    Parameters:
    image_path (str): The path to the directory where the images are stored.
    metadata_path (str): The path to the directory where the metadata will be saved.

    Returns:
    None
    """
    # Use the binary exifextract from include path
    binary = include_path + '/exifextract'
    command = [binary, images_path, metadata_path + '/metadata.csv']
    import subprocess
    # execute command
    popen = subprocess.Popen(command, stdout=subprocess.PIPE)
    popen.wait()

    # wait for the process to terminate
    output, error = popen.communicate()

    while popen.poll() is None:
        time.sleep(0.1)

    # check if the process terminated successfully
    if popen.returncode != 0:
        raise subprocess.CalledProcessError(popen.returncode, command)

    # load metadata from csv
    with open(metadata_path + '/metadata.csv', 'r') as f:
        reader = csv.reader(f)
        metadata = list(reader)
        header = metadata[0]

    metadata = metadata[1:]
    metadata_dict = {}
    for i, row in enumerate(metadata):
        metadata_dict[i] = {}
        for j in range(1, len(header)):
            metadata_dict[i][header[j]] = row[j]
        # add filename to metadata
        # remove ' from row[0]
        row[0] = row[0].replace("'", '')
        metadata_dict[i]['filename'] = row[0]


    # save metadata to database
    print("Generating SQL requests...")
    queries = gen_sql_requests(metadata_dict)
    # save to file queries.sql
    with open(metadata_path + '/queries.sql', 'w') as f:
        f.write(";\n".join(queries))


    print("Saving metadata to database...")
    # TODO: Save to database
    conn = create_server_connection(sql_host, sql_user, sql_password, sql_database)
    # Execute file to database
    cursor = conn.cursor()
    #cursor.execute("DROP TABLE IF EXISTS metadata")
    #cursor.execute("CREATE TABLE metadata (filename VARCHAR(255), key VARCHAR(255), value VARCHAR(255))")
    cursor.execute("LOAD DATA LOCAL INFILE '" + metadata_path + "/queries.sql' INTO TABLE metadata FIELDS TERMINATED BY ';' LINES TERMINATED BY '\n'")
    conn.commit()
    conn.close()

In [ ]:
asyncio.run(get_all_metadata(images_path))

# How to look at the metadata (sqlite format)
This is the way to look at the metadata information of an image in sqlite database format.

In [ ]:
conn = create_server_connection(sql_host, sql_user, sql_password, sql_database)

cursor = conn.cursor()
cursor.execute("SELECT * FROM metadata WHERE filename = 'image_0.jpg'")

result = cursor.fetchall()
for x in result:
    print(x)